##Setting up

In [1]:
import torch
# Check so there is a gpu available, a T4(free tier) is enough to run this notebook
assert (torch.cuda.is_available()==True)

In [6]:
#!pip install -e git+https://github.com/axolotl-ai-cloud/axolotl#egg=axolotl
#!git clone https://github.com/axolotl-ai-cloud/axolotl
!pip install -e git+"https://github.com/axolotl-ai-cloud/axolotl#egg=axolotl"
!cd axolotl

!pip3 install packaging ninja
#!pip3 install -e '.[flash-attn,deepspeed]'
!pip3 install flash-attn
!pip3 install deepspeed
#!pip install mamba-ssm

Obtaining axolotl from git+https://github.com/axolotl-ai-cloud/axolotl#egg=axolotl
  Cloning https://github.com/axolotl-ai-cloud/axolotl to ./src/axolotl
  Running command git clone --filter=blob:none --quiet https://github.com/axolotl-ai-cloud/axolotl /content/src/axolotl
  Resolved https://github.com/axolotl-ai-cloud/axolotl to commit e8d3da00814ec7773d33edd5643bb885d85686cb
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/lm-sys/FastChat.git (to revision 27a05b04a35510afb1d767ae7e5990cbd278f8fe) to /tmp/pip-install-yqziubun/fschat_0e901a96e4c24cee9538188f6d5e0597
  Running command git clone --filter=blob:none --quiet https://github.com/lm-sys/FastChat.git /tmp/pip-install-yqziubun/fschat_0e901a96e4c24cee9538188f6d5e0597
  Running command git rev-parse -q --verify 'sha^27a05b04a35510afb1d767ae7e5990cbd278f8fe'
  Running command git fetch -q https://github.com/lm-sys/FastChat.git 27a05b04a35510afb1d767ae7e5990cbd278f8fe
  Running command git checkout -q 27a05b04a3

###Hugging Face login

In [3]:
from huggingface_hub import notebook_login
notebook_login()

## Example configuration

In [7]:
import yaml

yaml_string = """
base_model: NousResearch/Meta-Llama-3.1-8B

load_in_8bit: true
load_in_4bit: false
strict: false

datasets:
  - path: tatsu-lab/alpaca
    type: alpaca
dataset_prepared_path: last_run_prepared
val_set_size: 0.05
output_dir: ./outputs/lora-out

sequence_len: 1024
sample_packing: true
pad_to_sequence_len: true

adapter: lora
lora_model_dir:
lora_r: 32
lora_alpha: 16
lora_dropout: 0.05
lora_target_linear: true
lora_fan_in_fan_out:
lora_modules_to_save:
  - embed_tokens
  - lm_head

wandb_project:
wandb_entity:
wandb_watch:
wandb_name:
wandb_log_model:

gradient_accumulation_steps: 8
micro_batch_size: 1
num_epochs: 1
optimizer: paged_adamw_8bit
lr_scheduler: cosine
learning_rate: 2e-5

train_on_inputs: false
group_by_length: false
bf16: auto
fp16:
tf32: false

gradient_checkpointing: true
gradient_checkpointing_kwargs:
  use_reentrant: false
early_stopping_patience:
resume_from_checkpoint:
logging_steps: 1
xformers_attention:
flash_attention: false
sdp_attention: true

warmup_steps: 1
evals_per_epoch: 2
eval_table_size:
saves_per_epoch: 1
debug:
deepspeed:
weight_decay: 0.0
fsdp:
fsdp_config:
special_tokens:
  pad_token: <|end_of_text|>
"""


# Convert the YAML string to a Python dictionary
yaml_dict = yaml.safe_load(yaml_string)

# Specify your file path
file_path = 'ft-8b-lora_colab.yaml'

# Write the YAML file
with open(file_path, 'w') as file:
    yaml.dump(yaml_dict, file)

Above we have a configuration file with base LLM model and datasets specified, amiong many other things. Axolotl can atuomatically detect whether the specifed datasets are on Hugguing Face repo or local machine.

The Axolotl configuration options encompass model and dataset selection, data pre-processing, and training. Let's go through them line by line:

*   "base model": String value, specifies the underlying pre-trained LLM that will be used for finetuning

Next we have options for model weights quantization. Quantization allows for reduction in occupied memory on GPUs.

*   "load_in_8bit": Boolean value, whether to quantize the model weights into 8-bit integer.

*   "load_in_4bit": Boolean value, whether to quantize the model weights into 4-bit integer.

*   "strict": Boolean value. If false, it allows for overidding established configuration options in the yaml file when executing in command-line interface.

*   "datasets": a list of dicts that contain path and type of data sets as well as other optional configurations where datasets are concerned. Supports multiple datasets.

*   "val_set_size": Either a float value less than one or an integer less than the total size of dataset. Sets the size of validation set from the whole dataset. If float, sets the proportion of the dataset assigned for validation. If integer, sets the direct size of validation set.

*   "output_dir": String value. Path of trained model.

For data preprocessing:

*   "sequence_len": Integer. Specifies the maximum sequence length of the input. Typically 2048 or less.

*   "pad_to_sequence_len": Boolean. Padding input to maximum sequence length.

*   "sample_packing": Boolean. Specifies whether to use multi-packing with block diagonal attention.

*   "special_tokens": Python dict, optional. Allows users to specify the additional special tokens to be ignored by the tokenizer.

For LoRA configuration and its hyperparamters:

*   "adapter": String. Either "lora" or "qlora", depending on user's choice.

*   "lora_model_dir": String, Optional. Path to directory that contains LoRA model, if there is already a trained LoRA model the user would like to use.

*   "lora_r": Integer. Refers to the rank of LoRA decomposition matrices. Higher value will reduce LoRA efficiency. Recommended to be set to 8.

*   "lora_alpha": Integer. Scale the weight matrices by $\frac{\text{lora_alpha}}{\text{lora_r}}$Recommended to be fixed at 16.

*   "lora_dropout": Float that is 1 or less. The dropout probability of a lora layer.

*   "lora_target_linear": Boolean. If true, lora will target all linear modules in the transformers architecture.

*   "lora_modules_to_save": If you added new tokens to the tokenizer, you may need to save some LoRA modules because they need to know the new tokens.

See [LoRA](https://arxiv.org/abs/2106.09685) for detailed explanation of LoRA implementation.

For the training configurations:

*   "gradient_accumulation_steps": Integer. The number of steps over which to accumulate gradient for batch training. E.g. if 2, backprop is performed every two steps.

*   "micro_batch_size": Integer. Batch size per gpu / gradient_accumulation_steps

*   "num_epochs": Integer. Number of epochs. One epoch is when training has looped over every batch in the whole data set once.

*   "optimizer": The optimizer to use for the training.

*   "learning_rate": The learning rate.

*   "lr_scheduler": The learning rate scheduler to use for adjusting learning rate during training.

*   "train_on_inputs": Boolean. Whether to ignore or include the user's prompt from the training labels.

*   "group_by_length": Boolean. Whether to group similarly sized data to minimize padding.

*   "bf16": Either "auto", "true", or "false". Whether to use CUDA bf16 floating point format. If set to "auto", will automatically apply bf16 should the gpu supports it.

*   "fp16": Optional. Specifies whether to use CUDA fp16. Automatically set to true if "bf16" is set to true. Otherwise false.

*   "tf32": Boolean. Whether to use CUDA tf32. Will override bf16.

*   "gradient_checkpointing": Boolean. Whether to use gradient checkpointing https://huggingface.co/docs/transformers/v4.18.0/en/performance#gradient-checkpointing

*   "gradient_checkpointing_kwargs": Python Dict. Fed into the trainer.

*   "logging_steps": Integer. Log training information over every specified number of steps.

*   "flash_attention": Boolean. Whether to use the [flash attention](https://github.com/Dao-AILab/flash-attention) mechanism.

*   "sdp_attention": Boolean. Whether to use the Scaled Dot Product attention mechanism (the attention mechanism in the [original implementation](https://arxiv.org/abs/1706.03762) of transformers.)

*   "warmup_steps": Integer. The number of pre-training steps where a very low learning rate is used.

*   "evals_per_epoch": Integer. Number of evaluations to be performed within one training epoch.

*   "saves_per_epoch": Integer. Number of times the model is saved in one training epoch.

*   "weight_decay": Positive Float. Sets the "strength" of weight decay (i.e. setting the coeeficient of L2 regularization)


The above is but a snippet aiming to get users familiarized with the types of streamlined configuration options axolotl provides. For a full list of configuration options, see [here](https://github.com/axolotl-ai-cloud/axolotl/blob/main/docs/config.qmd)


Train the model

In [11]:
!cd ./axolotl/
!accelerate launch -m axolotl.cli.train ./"ft-8b-lora_colab.yaml"

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
2024-10-09 16:03:07.667507: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-09 16:03:07.699692: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-09 16:03:07.709483: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has alread

Predict with trained model

In [ ]:
!cd ./src/axolotl
!accelerate launch -m axolotl.cli.inference "ft-8b_colab.yaml" --qlora_model_dir="./outputs/lora-out" --gradio

##Deeper Dive

It is also helpful to gain some familiarity over some of the core inner workings of axolotl

###Configuration Normalization

Axolotl uses a custom Dict class, called ```DictDefault```
to store configurations specified in the yaml configuration file (into a Python variable named ```cfg```). The definition for this custom Dict can be found in the [utils/dict.py](https://github.com/axolotl-ai-cloud/axolotl/blob/main/src/axolotl/utils/dict.py)

```DictDefault``` is amended such that calling a missing key from it will result in a ```None``` return type. This is important because if some configuration options aren't specified by the user, the '''None''' type allows Axolotl to perform boolean operations to determine the default settings for missing configurations. For more examples on how this is done, check out [utils/config/__init__.py](https://github.com/axolotl-ai-cloud/axolotl/blob/main/src/axolotl/utils/config/__init__.py)

###Loading Models, Tokenizers, and Trainer

If we inspect [cli.train.py](https://github.com/axolotl-ai-cloud/axolotl/blob/main/src/axolotl/cli/train.py), we will find that most of the heavy lifting were done by the function ```train()``` which is itself imported from [src/axolotl/train.py](https://github.com/axolotl-ai-cloud/axolotl/blob/main/src/axolotl/train.py).

```train()``` takes care of loading the appropriate tokenizer and pre-trained model through ```load_model()``` and ```load_tokenizer()``` from [src/axolotl/utils/models.py](https://github.com/axolotl-ai-cloud/axolotl/blob/main/src/axolotl/utils/models.py) respectively.

```load_tokenizer()``` loads in the appropriate tokenizer given the desired model, as well as chat templates.

```load_model()``` follows after tokenizer has been selected. It will automatically discern the base model type, load in the desired model, as well as applying model-appropriate attention mechanism modifications (e.g. flash attention). Depending on which base model the user chooses in the configuration, ```load_model()``` will utilize the corresponding "attention hijacking" script. For example, if the user specified the base model to be ```NousResearch/Meta-Llama-3.1-8B```, which is of llama type, and set ```flash_attn``` to ```True```, ```load_model()``` will load in [llama_attn_hijack_flash.py](https://github.com/axolotl-ai-cloud/axolotl/blob/main/src/axolotl/monkeypatch/llama_attn_hijack_flash.py). For a list of supported attention hijacking, please refer to the directory [/src/axolotl/monkeypatch/](https://github.com/axolotl-ai-cloud/axolotl/tree/main/src/axolotl/monkeypatch)

Another important operation encompassed in ```train()``` is setting up the training that takes into account of user-specified traning configurations (e.g. num_epochs, optimizer) through the use of ```setup_trainer()``` from [/src/axolotl/utils/trainer.py](https://github.com/axolotl-ai-cloud/axolotl/blob/main/src/axolotl/utils/trainer.py), which in turn relies on modules from [/src/axolotl/core/trainer_builder.py](https://github.com/axolotl-ai-cloud/axolotl/blob/main/src/axolotl/core/trainer_builder.py).
```trainer_builder.py``` provides a list of trainer object options bespoke for the task type (Causal or Reinforcement learning ('dpo', 'ipo', 'kto') )

###Monkey patch

The [Monkey patch directory](https://github.com/axolotl-ai-cloud/axolotl/tree/main/src/axolotl/monkeypatch) is where model architecture/optimization patching scripts are stored (these are modifications that are not implemented in the official releases, hence the name monkey patch). It includes attention jacking, ReLoRA, and unsloth optimization.